In [135]:
import cv2 as cv
import glob
import pandas as pd
from feat import Detector
import math
from pathlib import Path

from feat.plotting import imshow
from IPython.display import Image
import numpy as np

In [112]:
images = [name for name in glob.glob('data/DiffusionFER/DiffusionEmotion_S/cropped/*/*/*.png')]
detector = Detector(face_model="retinaface", landmark_model= "pfld") # , au_model = "xgb", emotion_model="resmasknet"
# tested woth mobilefacenet and pfld, pfld was faster and therefore chosen
# facial landmark detection works quite poorly for distorted faces.

In [148]:
example = 'data/DiffusionFER/DiffusionEmotion_S/cropped\\angry\\angry\\nnwotpxb_6.png'
example.split('/')[-2]
ex_img = cv.imread(example)
exf = detector.detect_faces(ex_img)
exlm = detector.detect_landmarks(ex_img, exf)
landmarks = exlm[0][0]
[math.dist(landmarks[33], landmark) for landmark in landmarks]
print(len(exf[0][0]))
image = cv.circle(ex_img, tuple(exlm[0][0][33].astype(int)), 2, [255,255,255], 3)
image = cv.polylines(ex_img, [exlm[0][0].astype(int)], True, (255, 0, 255))

imshow(image)

'DiffusionEmotion_S'

In [164]:
#features, face bounding box, feature landmarks

data = [extract_features(image, detector) for image in images]

In [167]:
data
fdata = list(filter(lambda sample: sample is not None, data))
fdata = pd.DataFrame(fdata)
fdata.to_csv("data/DiffusionFER/DiffusionEmotion_S/cropped/labels.csv")

In [162]:
# Concatenates the Euclidean distance from the tip of the nose, start and end of facebox as well as image path. If no face is found the sample is discarded.
def extract_features(path, detector):
    img = cv.imread(path)
    faces = detector.detect_faces(img)
    if(len(faces[0]) == 0): return
    landmarks = detector.detect_landmarks(img, faces)[0][0]
    features = [math.dist(landmarks[33], landmark) for landmark in landmarks] + [faces[0][0][2] - faces[0][0][0], faces[0][0][3] - faces[0][0][1]]
    features.append(path)
    features.append(path.split('/')[-2])
    return features